In [ ]:
!apt-get install bedtools

In [ ]:
!pip install umap-learn

In [ ]:
import pandas as pd
from google.colab import drive
import os
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
import matplotlib.pyplot as plt
import umap.umap_ as umap
import numpy as np

In [ ]:
drive.mount('/content/drive', force_remount=True)

In [ ]:
path = '/content/drive/MyDrive/Super_Enhancers/mm_project'

In [ ]:
samples = ['11_0077', '12_0118', '12_0449', '12_0450']

**Getting all enhancers data for each sample and concatenation into:**
- Single SE dataset
- Single TE dataset
- Single dataset consisting of unioned SE and TE datasets


In [ ]:
SE_table = pd.DataFrame()
TE_table = pd.DataFrame()
df_ste = pd.DataFrame()

for sample in samples:
  df_curr_se = pd.read_csv(f'{path}/Mouse_Lung_mm10/SE_{sample}_SE_mm10.bed', sep = '\t')
  df_curr_te = pd.read_csv(f'{path}/Mouse_Lung_mm10/SE_{sample}_TE_mm10.bed', sep = '\t')

  df_curr_se['region_length'] = df_curr_se.se_end - df_curr_se.se_start
  df_curr_te['region_length'] = df_curr_te.TE_end - df_curr_te.TE_start

  # handling corrupted data in TE dataset
  df_curr_te = df_curr_te[~df_curr_te.cell_id.isnull()]
  df_curr_te = df_curr_te[~df_curr_te.TE_cas_value.isnull()]
  df_curr_te['TE_con_value'] = df_curr_te.TE_con_value.astype('float')
  df_curr_te = df_curr_te[df_curr_te.TE_con_value < df_curr_te.TE_cas_value]
  df_curr_te = df_curr_te.drop_duplicates()

  df_curr_te['avg_rpm_diff'] = df_curr_te.TE_cas_value - df_curr_te.TE_con_value
  df_curr_se['avg_rpm_diff'] = df_curr_se.se_cas_value - df_curr_se.se_con_value

  df_ste_curr = pd.concat([df_curr_se[['cell_id', 'se_id','se_chr','se_start','se_end',
                                       'se_rank', 'region_length', 'avg_rpm_diff']].rename(columns = {'se_id' : 'ste_id',
                                                                                                      'se_rank': 'ste_rank',
                                                                                                      'se_chr' : 'ste_chr',
                                                                                                      'se_start' : 'ste_start',
                                                                                                      'se_end' : 'ste_end'}),
                           df_curr_te[['cell_id', 'TE_id', 'TE_chr','TE_start','TE_end',
                                       'TE_rank','region_length','avg_rpm_diff']].rename(columns = {'TE_id' : 'ste_id',
                                                                                                    'TE_rank' : 'ste_rank',
                                                                                                    'TE_chr' : 'ste_chr',
                                                                                                    'TE_start' : 'ste_start',
                                                                                                    'TE_end' : 'ste_end'})
                          ], ignore_index = True)

  SE_table = pd.concat([SE_table, df_curr_se], ignore_index=True)
  TE_table = pd.concat([TE_table, df_curr_te], ignore_index=True)
  df_ste = pd.concat([df_ste, df_ste_curr], ignore_index=True)

**Preparing single SE dataset for merge, merging SE and defining consolidated SE loci**

In [ ]:
df_se_concated = SE_table[['se_chr', 'se_start', 'se_end', 'cell_id', 'se_id', 'se_rank', 'region_length', 'avg_rpm_diff']]
df_se_concated[['se_chr', 'se_start', 'se_end', 'cell_id', 'se_id']].to_csv(f'{path}/Intermediate_tables/Set2__SE_concatinated.bed', index = False, sep = '\t', header = None)

In [ ]:
!sort -k1,1 -k2,2n {path}/Intermediate_tables/Set2__SE_concatinated.bed >  {path}/Intermediate_tables/sorted.Set2__SE_concatinated.bed
!bedtools merge -i {path}/Intermediate_tables/sorted.Set2__SE_concatinated.bed -c 4,4,5,5 -o count_distinct,collapse,count_distinct,collapse -d 12500 > {path}/Intermediate_tables/merged.sorted.Set2__SE_concatinated.bed

In [ ]:
df_merged = pd.read_csv(f'{path}/Intermediate_tables/merged.sorted.Set2__SE_concatinated.bed',
                        sep = '\t',
                        header = None,
                        names = ['se_chr', 'se_start', 'se_end', 'cell_id_count', 'cell_id_list', 'se_id_count',  'se_id_list'])


for s in samples:
  df_merged[f'SE_{s}'] = df_merged.cell_id_list.apply(lambda x: f'SE_{s}' in x).astype('int')

df_merged['se_locus_id'] =  ['se_region_' + str(i+1) for i in range(df_merged.shape[0])]
df_merged = df_merged[['se_locus_id', 'se_chr', 'se_start', 'se_end', 'se_id_list',
                        'SE_11_0077', 'SE_12_0118',
                        'SE_12_0449', 'SE_12_0450']].rename(columns = {'se_chr' : 'chr', 'se_start' : 'start', 'se_end' : 'end'})
df_merged.to_csv(f'{path}/Tables/Set2__Table1.csv', index = False)

In [ ]:
df_merged.head(5)

**Intersecting consolidated SE loci with unioned SE+TE datasets**

In [ ]:
df_merged[['chr', 'start', 'end', 'se_locus_id']].to_csv(f'{path}/Intermediate_tables/Set2__consolidated_SE_loci.bed', sep = '\t', index = False, header = False)

In [ ]:
# saving unioned SE and TE datasets
df_ste[['ste_chr', 'ste_start', 'ste_end',
        'cell_id', 'ste_id',  'ste_rank',
        'region_length', 'avg_rpm_diff']].to_csv(f'{path}/Intermediate_tables/Set2__SE_TE_concatinated.bed',
                                                   sep = '\t', index = False, header = False)

In [ ]:
!sort -k1,1 -k2,2n {path}/Intermediate_tables/Set2__SE_TE_concatinated.bed >  {path}/Intermediate_tables/sorted.Set2__SE_TE_concatinated.bed

In [ ]:
!bedtools intersect -a {path}/Intermediate_tables/sorted.Set2__SE_TE_concatinated.bed -b {path}/Intermediate_tables/Set2__consolidated_SE_loci.bed -wo -f 0.5 -F 1 -e > {path}/Intermediate_tables/Set2__STE_within_consolidated_SE_loci.bed

**Preparing a table of all enhancers signals, both SE and TE, within consolidated SE loci**

In [ ]:
df_ste_within_locus = pd.read_csv(f'{path}/Intermediate_tables/Set2__STE_within_consolidated_SE_loci.bed', sep = '\t', header = None,
                              names = ['ste_chr', 'ste_start', 'ste_end', 'cell_id',
                                       'ste_id',  'ste_rank', 'region_length', 'avg_rpm_diff',
                                       'chr', 'start', 'end', 'se_locus_id', 'overlap'] )

df_ste_within_locus = df_ste_within_locus[['se_locus_id', 'cell_id', 'ste_id', 'ste_chr', 'ste_start', 'ste_end',
                    'ste_rank', 'region_length', 'avg_rpm_diff', 'overlap']].merge(df_ste_within_locus.groupby(['se_locus_id', 'cell_id'],
                                                                                                               as_index = False)\
                                                                                                       .agg(locus_ste_overlap_total = ('overlap', np.sum)),
                                                                                   on = ['se_locus_id', 'cell_id'])
df_ste_within_locus['ste_weight_within_locus'] = df_ste_within_locus.overlap/df_ste_within_locus.locus_ste_overlap_total

In [ ]:
df_ste_within_locus = df_ste_within_locus[['se_locus_id', 'cell_id', 'ste_id',
                    'ste_chr', 'ste_start', 'ste_end', 'ste_rank',
                     'avg_rpm_diff', 'overlap', 'ste_weight_within_locus']]

df_ste_within_locus.to_csv(f'{path}/Tables/Set2__Table2.csv', index = False)

**Defining peak (maximum) and weighted average activity within all consolidated SE locus for each sample separately**

In [ ]:
df_ste_within_locus['is_SE'] = df_ste_within_locus.ste_id.str.contains('SE').astype('int')
df_ste_within_locus['weighted_avg_rpm_diff'] = df_ste_within_locus.avg_rpm_diff * df_ste_within_locus.ste_weight_within_locus
df_locus_activity = df_merged[['se_locus_id']].merge(df_ste_within_locus.groupby(['se_locus_id', 'cell_id'], as_index = False)\
                                                                         .agg(avg_rpm_diff__max = ('avg_rpm_diff', 'max'),
                                                                              avg_rpm_diff__weighted = ('weighted_avg_rpm_diff', sum ),
                                                                              max_rank = ('ste_rank', 'max'),
                                                                              min_rank = ('ste_rank', 'min'),
                                                                              active_SE = ('is_SE', 'max'),
                                                                              active_SE_count = ('is_SE',  lambda x : np.sum( x) ),
                                                                              active_TE_count = ('is_SE', lambda x : np.sum(1 - x) )),
                                                       on = 'se_locus_id')

In [ ]:
df_locus_activity.to_csv(f'{path}/Tables/Set2__Table3.csv', index = False)

**Feature Matrix Construction**

In [ ]:
df_features = pd.pivot_table(df_locus_activity, index = 'se_locus_id', columns = 'cell_id', values = 'avg_rpm_diff__max')

#sort loci
df_features['se_locus_num'] =  df_features.index.str.split('se_region_').str.get(1).astype('int')
df_features = df_features.sort_values('se_locus_num')
df_features.drop(columns='se_locus_num', inplace = True)

In [ ]:
df_signal_max = df_locus_activity[['se_locus_id','cell_id','avg_rpm_diff__max']].copy()
df_signal_max['consolidated_SE_locus_signal'] = df_signal_max.avg_rpm_diff__max
df_signal_max['method'] = 'peak_activity'

df_signal_weighted = df_locus_activity[['se_locus_id','cell_id','avg_rpm_diff__weighted']].copy()
df_signal_weighted['consolidated_SE_locus_signal'] = df_signal_weighted.avg_rpm_diff__weighted
df_signal_weighted['method'] = 'weighted_avg_activity'

df_signal = pd.concat([df_signal_max, df_signal_weighted], ignore_index = True)

In [ ]:
#Visually, the difference in signal distribution between the two approaches is not noticeable.
# However, both of these approaches are required for deep analysis of a specific locus activity
sns.boxplot(df_signal, y='cell_id', x='consolidated_SE_locus_signal', hue = 'method')

In [ ]:
(df_features.isnull()).sum()/df_features.shape[0]

In [ ]:
df_ste_within_locus[df_ste_within_locus.is_SE == 1].groupby('cell_id', as_index = False).agg(se_num = ('ste_rank', 'max'))

**Feature processing**

In [ ]:
for col in df_features.columns:
  df_features[f'{col}_medianNormalized'] = df_features[col]/df_features[df_features[col].notnull()][col].median()

- Global **data imputation** with a small positive value

In [ ]:
imputation_value = 1e-06  #
X_medianNormalized_imputed = np.nan_to_num(df_features.iloc[:, 4:].values, nan=imputation_value)
df_features[df_features.columns[4:] + '_imputed'] = X_medianNormalized_imputed.copy()

- Log(1+X) data transformation

In [ ]:
X_log1p = np.log1p(df_features.iloc[:, 8:].values)
df_features[df_features.columns[8:] + '_log1p'] = X_log1p.copy()

- z-scaling

In [ ]:
ss = StandardScaler()
ss.fit(X_log1p)

X_zscaled = ss.transform(X_log1p)
df_features[df_features.columns[12:] + '_zscaled'] = X_zscaled.copy()

In [ ]:
df_features.reset_index(inplace=True)

In [ ]:
df_features = df_features.merge(df_merged[['se_locus_id', 'SE_11_0077', 'SE_12_0118',
                                           'SE_12_0449', 'SE_12_0450']] ,
                   on = 'se_locus_id', suffixes = ('', '_is'))

In [ ]:
feature_columns = list(df_features.columns)

In [ ]:
feature_columns_ordered = feature_columns[0:1] + feature_columns[21:] + feature_columns[1:21]

In [ ]:
df_features[feature_columns_ordered].to_csv(f'{path}/Tables/Set2__Table4.csv', index = False)